# Jigsaw Puzzle — Milestone 1 Pipeline

This notebook implements a full Milestone-1 pipeline for the puzzle dataset located at `/mnt/data/1.jpg`.

**Features implemented:**
- Automatic grid detection (projection profiles) with fallback to manual grid size
- Tile extraction for 2x2, 4x4, 8x8 puzzles
- Per-tile enhancement (denoise, CLAHE, optional resize)
- Binary mask creation and contour extraction
- Artifacts saving (enhanced images, masks, contours, metadata)
- Visualization helpers (before/after and grid overlay)

Run each cell in order. Modify parameters near the top if needed.


In [ ]:

# Imports and parameters
import cv2, os, json, numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

INPUT_PATH = "D:/Image-Processing-Project/Gravity_Falls/puzzle_2x2/1.jpg"   # representative image
OUT_DIR = "D:/Image-Processing-Project/output"
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUT_DIR,"tiles"), exist_ok=True)
os.makedirs(os.path.join(OUT_DIR,"visualizations"), exist_ok=True)

# Parameters (tweak if needed)
FALLBACK_GRID = None   # set to (rows,cols) if auto-detect fails, e.g. (4,4)
PADDING = 4            # pixels to include around tiles when cropping
RESIZE_TO = None       # set e.g. (256,256) to resize tiles for uniformity, or None to keep original
SAVE_CONTOUR_NPY = True
